## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Farooq and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Farooq! Congratulations on being a Chief AI Engineer - that's a impressive title! What kind of AI projects are you currently working on? Are you interested in exploring new AI applications or improving existing ones? I'm here to help with any AI-related questions or discussions you'd like to have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 25, 'total_tokens': 91, 'completion_time': 0.055, 'prompt_time': 0.003452561, 'queue_time': 0.010288889, 'total_time': 0.058452561}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3a8d7bd-5867-44d7-9723-39dd4c067602-0', usage_metadata={'input_tokens': 25, 'output_tokens': 66, 'total_tokens': 91})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Farooq and I am a Chief AI Engineer"),
        AIMessage(content="Hello Farooq! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Easy one! Your name is Farooq, and you're a Chief AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 96, 'total_tokens': 115, 'completion_time': 0.015833333, 'prompt_time': 0.032408093, 'queue_time': 0.384726216, 'total_time': 0.048241426}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-9dca81f7-a025-491a-bea6-2c55b1c59032-0', usage_metadata={'input_tokens': 96, 'output_tokens': 19, 'total_tokens': 115})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Farooq and I am a Chief AI Engineer")],
    config=config
)

In [9]:
response.content

"Nice to meet you, Farooq! Congratulations on being a Chief AI Engineer! That's a fascinating role. What kind of AI projects have you worked on in the past, and what are you currently focused on? I'm curious to learn more about your experience and expertise in the field of artificial intelligence."

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="I remember! Your name is Farooq, and you're a Chief AI Engineer!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 102, 'total_tokens': 121, 'completion_time': 0.015833333, 'prompt_time': 0.017570515, 'queue_time': 0.001188505000000003, 'total_time': 0.033403848}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a5311b8-67bc-4823-9fc5-1697c1e3bc23-0', usage_metadata={'input_tokens': 102, 'output_tokens': 19, 'total_tokens': 121})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I apologize, but I'm a large language model, I don't have the ability to know your name unless you've told me or provided it to me in some way. I'm a new conversation each time you interact with me, so I don't retain any information about our previous conversations. If you'd like to share your name with me, I'd be happy to learn it!"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Nice to meet you, John! I'm glad you shared your name with me. I'll remember it for our conversation today. How can I assist you, John? Do you have any questions, topics you'd like to discuss, or games you'd like to play? I'm all ears!"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I remember! Your name is John!'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Farooq")]})

AIMessage(content="Nice to meet you, Farooq! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 38, 'total_tokens': 80, 'completion_time': 0.035, 'prompt_time': 0.005005392, 'queue_time': 0.008968338, 'total_time': 0.040005392}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-a06efaa4-888b-492d-a7f5-32c277225904-0', usage_metadata={'input_tokens': 38, 'output_tokens': 42, 'total_tokens': 80})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Farooq")],
    config=config
)

response

AIMessage(content="Nice to meet you, Farooq! It's great to have you here. How can I assist you today? Do you have any specific questions or topics you'd like to discuss? I'm all ears!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 38, 'total_tokens': 82, 'completion_time': 0.036666667, 'prompt_time': 0.004994352, 'queue_time': 0.009280447, 'total_time': 0.041661019}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5e533fc-c1d2-4f6e-86d4-2088b592d40e-0', usage_metadata={'input_tokens': 38, 'output_tokens': 44, 'total_tokens': 82})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Farooq!'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"French"})
response.content

'Bonjour Krish ! Enchanté de faire votre connaissance ! (Hello Krish! Nice to meet you!)'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Farooq ")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते! मैं आपकी मदद करने के लिए यहाँ हूँ, फारूफ! क्या आपके लिए कुछ है? (Namaste! Main aapki madad karne ke liye yahan hoon, Farooq! Kya aapke liye kuch hai?)'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [24]:
response.content

'आपका नाम फारूफ है! (Aapka naam Farooq hai!)'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    # Set the maximum number of tokens allowed in the trimmed message list.
    max_tokens=45,

    # Define the trimming strategy. "last" keeps the most recent messages within the token limit.
    strategy="last",

    # Specify the function used to count tokens in the messages. This is typically model-specific,
    # e.g., a tokenizer for OpenAI models like GPT.
    token_counter=model,

    # Include the initial SystemMessage in the trimmed messages, even if trimming occurs.
    include_system=True,

    # Do not allow partial messages. Each message must be included fully or excluded entirely.
    allow_partial=False,

    # Start the trimming process at the first human message in the conversation.
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\DS Learning Material\Gen AI Course\Chatbots\genvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\DS Learning Material\Gen AI Course\Chatbots\genvenv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SCM\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I'm just an assistant, I don't know your personal preferences, but I can suggest some popular flavors if you'd like!"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me the math problem: "whats 2 + 2"'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I apologize, but I don't know your name. I'm a new conversation partner, and we just started talking! Would you like to introduce yourself?"

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a math problem yet! This conversation just started, and I'm ready to help with any questions you have, including math problems. What's on your mind?"